<a href="https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr-lease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Environment**

## **Bootstrap public repo**

In [11]:
from pathlib import Path

if not Path(f"/content").is_dir():
  %cd ~
else:
  %cd /content

if not Path(f"src/jams-pub").is_dir():
  !git clone https://github.com/clarkde5/jams-pub src/jams-pub
else:
  print("src/jams-pub already exists")

/home/dclark
src/jams-pub already exists


## **Pull private resources**

In [17]:
runOptional = True
def CallPrivateEnvSetup():
  from pathlib import Path

  home = str(Path.home())

  try:
    from google.colab import userdata
    try:
      rsa_private_key = userdata.get('PrivateKey')
    except:
      rsa_private_key = ""
      print("PrivateKey is required, but not found in secrets / colab userdata")
      return
  except:
    print("google.colab could not be loaded")
    if not Path(f"{home}/.ssh/id_rsa").is_file():
      print("rsa key does not exist, please create manually")
      return
    else:
      print("rsa key already exists")

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    !python src/jams-pub/env-setup.py "{rsa_private_key}"

  if not Path(f"{home}/.ssh/id_rsa").is_file():
    print("env-setup failed to create rsa key")
    return
  else:
    #!apt-get install dos2unix -y
    #!dos2unix ~/.ssh/id_rsa
    !sed -i 's/\r$//' ~/.ssh/id_rsa

  if not Path(f"src/jams").is_dir():
    !git clone git@github.com:clarkde5/jams.git src/jams
  else:
    print("src/jams already exists")

if __name__ == "__main__":
  if runOptional == True:
    CallPrivateEnvSetup()
  else:
    print("Not running optional cell")

google.colab could not be loaded
rsa key already exists
Cloning into 'src/jams'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 242 (delta 5), reused 9 (delta 1), pack-reused 221
Receiving objects: 100% (242/242), 76.54 MiB | 12.31 MiB/s, done.
Resolving deltas: 100% (7/7), done.


# **Parse Lease Output**

**NOTE:** Requires [Pull private resources](https://colab.research.google.com/github/clarkde5/jams-pub/blob/main/colabs/jams-env-doctr.ipynb#scrollTo=CyQkr3hoftql)

In [13]:
runOptional = True
def getContractsForPage(page_idx,page):
  import re
  CurrentFound = page_idx != 0
  contracts = []

  for block in page["blocks"]:
    for line in block["lines"]:
      for word_idx,word in enumerate(sorted(line["words"], key = lambda x: x["geometry"][0][1])):
        if re.search("CURRENT",word["value"]):
          CurrentFound = True

        if not CurrentFound:
          continue

        if re.search("\d{3}-\d{7}-\d{3}",word["value"]):
          contracts.append({"contract_number": word["value"], "pdf_y": word["geometry"][0][1]+page_idx, "page": page_idx+1})
        else:
          continue

  return contracts

def getSerialNumbersForPage(page_idx,page):
  import re
  CurrentFound = page_idx != 0
  serialNumbers = []

  for block in page["blocks"]:
    for line in block["lines"]:
      for word_idx,word in enumerate(sorted(line["words"], key = lambda x: x["geometry"][0][1])):
        if re.search("CURRENT",word["value"]):
          CurrentFound = True

        if not CurrentFound:
          continue

        if re.search("SERIAL",word["value"]):
          serial_number_word = line["words"][2]
          serialNumbers.append({"serial_number": serial_number_word["value"], "pdf_y": serial_number_word["geometry"][0][1]+page_idx, "page": page_idx+1})
        else:
          continue

  return serialNumbers

def getPaymentDue(page_idx,page):
  import re
  CurrentFound = page_idx != 0
  PleaseFound = False
  prices = []

  for block in page["blocks"]:
    for line in block["lines"]:
      for word_idx,word in enumerate(sorted(line["words"], key = lambda x: x["geometry"][0][1])):
        if re.search("PLEASE",word["value"]):
          PleaseFound = True

        if not PleaseFound:
          continue

        if re.search("CURRENT",word["value"]):
          CurrentFound = True

        if not CurrentFound:
          continue

        if re.search("-{0,1}\d+\.\d{2}",word["value"]):
          prices.append({"price": word["value"], "pdf_y": word["geometry"][0][1]+page_idx, "page": page_idx+1})
        else:
          continue

  return prices

def main():
  import json
  import re

  f = open('src/jams/output/Aug 23 Lease-docTR.json')
  data = json.load(f)

  contracts = []
  serialNumbers = []
  paymentDue = []

  for page_idx,page in enumerate(data["pages"]):
    contracts += getContractsForPage(page_idx,page)
    serialNumbers += getSerialNumbersForPage(page_idx,page)
    paymentDue += getPaymentDue(page_idx,page)

  contractSerials = contracts + serialNumbers + paymentDue
  contractSerials = sorted(contractSerials, key = lambda x: x["pdf_y"])
  currentPage = 0

  for contractSerialPair in contractSerials:
    if contractSerialPair["page"] != currentPage:
      currentPage = contractSerialPair["page"]
      print("\r\nPage: " + str(currentPage) + "\r\n")

    if "contract_number" in contractSerialPair:
      print("Contract Number: " + contractSerialPair["contract_number"])
    elif "serial_number" in contractSerialPair:
      print("Serial Number: " + contractSerialPair["serial_number"])
    elif "price" in contractSerialPair:
        print("Due: " + contractSerialPair["price"])
    else:
      print("Error unknown pair: " + contractSerialPair)

if __name__ == "__main__":
  if runOptional == True:
    main()
  else:
    print("Not running optional cell")


Page: 1

Contract Number: 500-0245168-184
Serial Number: AAFNO11
Due: 32.85
Due: 2.55

Page: 2

Contract Number: 500-0245168-195
Serial Number: AA2K011002376
Due: 173.72
Due: 13.95
Contract Number: 500-0245168-196
Serial Number: AAFJ011002153
Due: 25.64
Due: 2.06
Contract Number: 500-0245168-197
Serial Number: AA2K011003939
Due: 173.72
Due: 10.42

Page: 3

Contract Number: 500-0245168-198
Serial Number: AA2K011005668
Due: 173.72
Due: 14.33
Contract Number: 500-0245168-202
Serial Number: AA2J011006646
Due: 205.04
Due: 12.31
Contract Number: 500-0245168-203
Serial Number: AA2K011006901
Due: 181.37
Due: 16.09
Contract Number: 500-0245168-204
Serial Number: AAFJ011002440
Due: 25.64
Due: 1.54
Contract Number: 500-0245168-205
Serial Number: AAJN011001635
Due: 64.79
Due: 5.75
Contract Number: 500-0245168-206
Serial Number: AA2K011008138
Due: 173.96
Due: 13.92

Page: 4

Contract Number: 500-0245168-206
Serial Number: AA2J011010939
Due: 205.04
Due: 16.40
Contract Number: 500-0245168-208
Serial